# LLM Optimization Modelling Experiment

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [595]:
problem = '''You are the person in charge of packing in a large company. Your job is to skillfully pack items of various weights in a box with predetermined capacity. The objective is to use as few boxes as possible. There is a set of items and a set of boxes with an upper bound on the amount of boxes. Each of the items has a known weight. Each item is packed into one box and items are packed into the chosen boxes. Please formulate a mathematical optimization model for this problem.   '''

## 2. Ask for parameters

In [609]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [610]:
#Show the resopnse in a formatted way
Markdown(response.text)

### Sets:
*  $I$: Set of items.
*  $J$: Set of boxes.

### Parameters:
*  $w_i$: Weight of item $i \in I$.
*  $C$: Capacity of each box $j \in J$.
*  $B$: Upper bound on the number of boxes that can be used. 

### Decision Variables:
*  $x_{ij}$: Binary variable equal to 1 if item $i \in I$ is assigned to box  $j \in J$, and 0 otherwise.
*  $y_j$: Binary variable equal to 1 if box $j \in J$ is used, and 0 otherwise. 


# 2. Ask for objective

In [611]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [612]:
Markdown(response2.text)

Minimize $\sum_{j \in J} y_j$ 


# 3. Ask for constraints

In [613]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [614]:
Markdown(response3.text)

Subject to:

* **Capacity constraint:** $\sum_{i \in I} w_i x_{ij} \le C y_j  $  for all $j \in J$ 
* **Assignment constraint:**  $\sum_{j \in J} x_{ij} = 1$ for all $i \in I$
* **Box limit constraint:** $\sum_{j \in J} y_j \le B$
* **Binary constraints:** $x_{ij} \in \{0,1\}$ for all $i \in I$, $j \in J$ and $y_j \in \{0,1\}$ for all $j \in J$. 


## 3. Generate the pyomo code

In [615]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [616]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample data (Replace with your actual data)
items = ['A', 'B', 'C', 'D']  
boxes = ['Box1', 'Box2', 'Box3']
weight = {'A': 4, 'B': 2, 'C': 5, 'D': 3} 
capacity = 10 
box_limit = 2

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=items) 
model.J = pyo.Set(initialize=boxes)

# Parameters
model.w = pyo.Param(model.I, initialize=weight)
model.C = pyo.Param(initialize=capacity)
model.B = pyo.Param(initialize=box_limit)

# Variables
model.x = pyo.Var(model.I, model.J, within=pyo.Binary) 
model.y = pyo.Var(model.J, within=pyo.Binary)

# Objective function
def objective_rule(model):
    return sum(model.y[j] for j in model.J)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
def capacity_constraint(model, j):
    return sum(model.w[i] * model.x[i, j] for i in model.I) <= model.C * model.y[j]
model.capacity = pyo.Constraint(model.J, rule=capacity_constraint)

def assignment_constraint(model, i):
    return sum(model.x[i, j] for j in model.J) == 1
model.assignment = pyo.Constraint(model.I, rule=assignment_constraint)

def box_limit_constraint(model):
    return sum(model.y[j] for j in model.J) <= model.B
model.box_limit = pyo.Constraint(rule=box_limit_constraint)

# Solve
solver = pyo.SolverFactory('glpk')  
results = solver.solve(model)

# Print results
print("Status:", results.solver.status)
print("Termination condition:",results.solver.termination_condition)
print("Optimal objective value:", pyo.value(model.obj))

# Print box assignments
for j in model.J:
    if pyo.value(model.y[j]) > 0:
        print(f"Items in {j}:", [i for i in model.I if pyo.value(model.x[i, j]) > 0]) 
```

**Explanation:**

1. **Import Pyomo:**  `import pyomo.environ as pyo` imports the Pyomo library.
2. **Sample Data:**  The code includes sample data for items, boxes, weights, capacity, and the box limit. You should replace this with your actual data.
3. **Model Initialization:** `model = pyo.ConcreteModel()` creates a concrete Pyomo model.
4. **Sets:**  Sets `model.I` (items) and `model.J` (boxes) are defined using the sample data.
5. **Parameters:**  Parameters `model.w` (weights), `model.C` (capacity), and `model.B` (box limit) are defined and initialized.
6. **Variables:** 
   - `model.x` is a binary variable representing item-to-box assignments.
   - `model.y` is a binary variable indicating if a box is used.
7. **Objective Function:** The `objective_rule` function defines the objective to minimize the number of boxes used (`sum(model.y[j] for j in model.J)`).
8. **Constraints:**
   - `capacity_constraint`: Ensures that the total weight in a box does not exceed its capacity.
   - `assignment_constraint`: Ensures that each item is assigned to exactly one box.
   - `box_limit_constraint`: Limits the total number of boxes used.
9. **Solver:** A solver (GLPK in this case) is selected using `pyo.SolverFactory('glpk')`.
10. **Solving and Results:** The model is solved, and the status, termination condition, and optimal objective value are printed. The code then prints the assignment of items to boxes. 


## 4. Input problem data and try running the generated code

In [618]:


import pyomo.environ as pyo

# Sample data (Replace with your actual data)
items = ['A', 'B', 'C', 'D']  
boxes = ['Box1', 'Box2', 'Box3']
weight = {'A': 4, 'B': 2, 'C': 5, 'D': 3} 
capacity = 10 
box_limit = 2

items =  range(1,25)
weight = {1: 2,2: 2,3: 2,4: 2,5: 3,6: 3,7:4,8: 4,9: 4,10: 4,11: 4,12:4,13: 5,14:5,15: 5,16: 5,17: 5,18: 5,19: 6,20:6,21: 7,22: 7,23: 8, 24:8} # Weight of each item
boxes = range(1,15)
capacity = 9
box_limit = 15

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=items) 
model.J = pyo.Set(initialize=boxes)

# Parameters
model.w = pyo.Param(model.I, initialize=weight)
model.C = pyo.Param(initialize=capacity)
model.B = pyo.Param(initialize=box_limit)

# Variables
model.x = pyo.Var(model.I, model.J, within=pyo.Binary) 
model.y = pyo.Var(model.J, within=pyo.Binary)

# Objective function
def objective_rule(model):
    return sum(model.y[j] for j in model.J)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
def capacity_constraint(model, j):
    return sum(model.w[i] * model.x[i, j] for i in model.I) <= model.C * model.y[j]
model.capacity = pyo.Constraint(model.J, rule=capacity_constraint)

def assignment_constraint(model, i):
    return sum(model.x[i, j] for j in model.J) == 1
model.assignment = pyo.Constraint(model.I, rule=assignment_constraint)

def box_limit_constraint(model):
    return sum(model.y[j] for j in model.J) <= model.B
model.box_limit = pyo.Constraint(rule=box_limit_constraint)

# Solve
solver = pyo.SolverFactory('glpk')  
results = solver.solve(model)

# Print results
print("Status:", results.solver.status)
print("Termination condition:",results.solver.termination_condition)
print("Optimal objective value:", pyo.value(model.obj))

Status: ok
Termination condition: optimal
Optimal objective value: 13.0


## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [619]:
print(response.text)

### Sets:
*  $I$: Set of items.
*  $J$: Set of boxes.

### Parameters:
*  $w_i$: Weight of item $i \in I$.
*  $C$: Capacity of each box $j \in J$.
*  $B$: Upper bound on the number of boxes that can be used. 

### Decision Variables:
*  $x_{ij}$: Binary variable equal to 1 if item $i \in I$ is assigned to box  $j \in J$, and 0 otherwise.
*  $y_j$: Binary variable equal to 1 if box $j \in J$ is used, and 0 otherwise. 



In [620]:
print(response2.text)

Minimize $\sum_{j \in J} y_j$ 



In [621]:
print(response3.text)

Subject to:

* **Capacity constraint:** $\sum_{i \in I} w_i x_{ij} \le C y_j  $  for all $j \in J$ 
* **Assignment constraint:**  $\sum_{j \in J} x_{ij} = 1$ for all $i \in I$
* **Box limit constraint:** $\sum_{j \in J} y_j \le B$
* **Binary constraints:** $x_{ij} \in \{0,1\}$ for all $i \in I$, $j \in J$ and $y_j \in \{0,1\}$ for all $j \in J$. 



In [622]:
print(response4.text)

```python
import pyomo.environ as pyo

# Sample data (Replace with your actual data)
items = ['A', 'B', 'C', 'D']  
boxes = ['Box1', 'Box2', 'Box3']
weight = {'A': 4, 'B': 2, 'C': 5, 'D': 3} 
capacity = 10 
box_limit = 2

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=items) 
model.J = pyo.Set(initialize=boxes)

# Parameters
model.w = pyo.Param(model.I, initialize=weight)
model.C = pyo.Param(initialize=capacity)
model.B = pyo.Param(initialize=box_limit)

# Variables
model.x = pyo.Var(model.I, model.J, within=pyo.Binary) 
model.y = pyo.Var(model.J, within=pyo.Binary)

# Objective function
def objective_rule(model):
    return sum(model.y[j] for j in model.J)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
def capacity_constraint(model, j):
    return sum(model.w[i] * model.x[i, j] for i in model.I) <= model.C * model.y[j]
model.capacity = pyo.Constraint(model.J, rule=capacity_constraint)

def assignment_constraint(model, i):
  